# Hybrid Classical Quantum AutoEncoder for anomaly detection

## VQC
The VQC implemented is the number 10 of the reference paper, which essentially is a stack Pauli $YX$ rotation gate and a circular series of controlled $CX$, stacked while alternated with an encoding based on the Pauli $X$ rotation

In [12]:
from qiskit.circuit import QuantumCircuit, Gate, ParameterVector
def get_encoding_block(nqubits: int, features: list[float]) -> Gate:
    """
    need n features
    """
    assert nqubits == len(features)
    block = QuantumCircuit(nqubits, name="Encoding Block")
    for i in range(nqubits):
        block.rx(features[i], i)
    return block.to_gate()

def get_ansatz_block(nqubits: int, parameters: ParameterVector) -> Gate:
    """
    need 2n parameters
    """
    assert nqubits * 2 == len(parameters)
    block = QuantumCircuit(nqubits, name="Ansatz Block")
    for i in range(nqubits):
        block.ry(parameters[i], i)
        block.rx(parameters[i + nqubits], i)
    if nqubits > 1:
        block.cx(nqubits - 1, 0)
        for i in range(nqubits - 1):
            block.cx(i, i + 1)
    return block.to_gate()

def get_ansatz(nqubits: int, parameters: ParameterVector, features: list[float], reps=3) -> QuantumCircuit:
    assert len(parameters) == reps * 2 * nqubits
    ansatz = QuantumCircuit(nqubits)
    ansatz.compose(get_ansatz_block(nqubits, parameters[:2 * nqubits]), range(nqubits), inplace=True)
    for i in range(1, reps):
        ansatz.barrier()
        ansatz.compose(get_encoding_block(nqubits, features), range(nqubits), inplace=True)
        ansatz.barrier()
        ansatz.compose(get_ansatz_block(nqubits, parameters[2 * nqubits * i :2 * nqubits * (i + 1)]), range(nqubits), inplace=True)
    return ansatz


In [14]:
features = [0] * 4
pv = ParameterVector("theta", 8 * 3)
qc = get_ansatz(4, pv, features)
qc.decompose().draw()

┌──────────────┐┌──────────────┐┌───┐                ░ ┌───────┐ ░ »
q_0: ┤ Ry(theta[0]) ├┤ Rx(theta[4]) ├┤ X ├──■─────────────░─┤ Rx(0) ├─░─»
     ├──────────────┤├──────────────┤└─┬─┘┌─┴─┐           ░ ├───────┤ ░ »
q_1: ┤ Ry(theta[1]) ├┤ Rx(theta[5]) ├──┼──┤ X ├──■────────░─┤ Rx(0) ├─░─»
     ├──────────────┤├──────────────┤  │  └───┘┌─┴─┐      ░ ├───────┤ ░ »
q_2: ┤ Ry(theta[2]) ├┤ Rx(theta[6]) ├──┼───────┤ X ├──■───░─┤ Rx(0) ├─░─»
     ├──────────────┤├──────────────┤  │       └───┘┌─┴─┐ ░ ├───────┤ ░ »
q_3: ┤ Ry(theta[3]) ├┤ Rx(theta[7]) ├──■────────────┤ X ├─░─┤ Rx(0) ├─░─»
     └──────────────┘└──────────────┘               └───┘ ░ └───────┘ ░ »
«      ┌──────────────┐┌───────────────┐┌───┐                ░ ┌───────┐ ░ »
«q_0: ─┤ Ry(theta[8]) ├┤ Rx(theta[12]) ├┤ X ├──■─────────────░─┤ Rx(0) ├─░─»
«      ├──────────────┤├───────────────┤└─┬─┘┌─┴─┐           ░ ├───────┤ ░ »
«q_1: ─┤ Ry(theta[9]) ├┤ Rx(theta[13]) ├──┼──┤ X ├──■────────░─┤ Rx(0) ├─░─»
«     ┌┴──────────────┤├───────────────┤  │  └───┘┌─┴─┐      ░ ├───────┤ ░ »
«q_2: ┤ Ry(theta[10]) ├┤ Rx(theta[14]) ├──┼───────┤ X ├──■───░─┤ Rx(0) ├─░─»
«     ├───────────────┤├───────────────┤  │       └───┘┌─┴─┐ ░ ├───────┤ ░ »
«q_3: ┤ Ry(theta[11]) ├┤ Rx(theta[15]) ├──■────────────┤ X ├─░─┤ Rx(0) ├─░─»
«     └───────────────┘└───────────────┘               └───┘ ░ └───────┘ ░ »
«     ┌───────────────┐┌───────────────┐┌───┐               
«q_0: ┤ Ry(theta[16]) ├┤ Rx(theta[20]) ├┤ X ├──■────────────
«     ├───────────────┤├───────────────┤└─┬─┘┌─┴─┐          
«q_1: ┤ Ry(theta[17]) ├┤ Rx(theta[21]) ├──┼──┤ X ├──■───────
«     ├───────────────┤├───────────────┤  │  └───┘┌─┴─┐     
«q_2: ┤ Ry(theta[18]) ├┤ Rx(theta[22]) ├──┼───────┤ X ├──■──
«     ├───────────────┤├───────────────┤  │       └───┘┌─┴─┐
«q_3: ┤ Ry(theta[19]) ├┤ Rx(theta[23]) ├──■────────────┤ X ├
«     └───────────────┘└───────────────┘               └───┘